In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix,precision_recall_curve,f1_score,precision_score,recall_score,auc,roc_auc_score,roc_curve,recall_score,classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore")
np.random.seed(42)

/home/inderjeet78/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv("creditcard.csv")
data['normAmount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))
data = data.drop(['Time','Amount'],axis=1)
X = data.iloc[:,data.columns != 'Class']
Y = data.iloc[:,data.columns == 'Class']
pca = PCA(n_components=25)
X = pca.fit_transform(X)
X_train,testX,Y_train,testY = train_test_split(X,Y,test_size=0.20,random_state=21, stratify=Y)
X_train,valX,Y_train,valY = train_test_split(X_train,Y_train,test_size=0.20,random_state=21, stratify=Y_train)

In [3]:
X_train = np.array(X_train)
valX = np.array(valX)
testX = np.array(testX)
Y_train = np.array(Y_train)
valY = np.array(valY)
testY = np.array(testY)

# Cross Validation

In [4]:
f1 = []
for m in range(1,25):
    for n in range(50,100):
        clf = IsolationForest(n_estimators=n,max_features=m,contamination=0.010,random_state=21)
        clf.fit(X_train)
        scores_pred = clf.decision_function(X_train)
        y_pred=clf.predict(valX)
        y_pred[y_pred==1]=0
        y_pred[y_pred==-1]=1
        f = f1_score(valY,y_pred)
        f1.append(f)
print("Best f-1 Score 10/90 for Isolation Forest is : ",max(f1) )
print("Best f-1 Score 10/90 for Isolation Forest parameters grid no. : ", f1.index(max(f1))+1) 

Best f-1 Score 10/90 for Isolation Forest is :  0.21682242990654207
Best f-1 Score 10/90 for Isolation Forest parameters grid no. :  4


# Test Set Predictions

In [5]:
clf = IsolationForest(n_estimators=69,max_features=1,contamination=0.010,random_state=21)
clf.fit(X_train)
y_pred1 = clf.predict(valX)
y_pred2 = clf.predict(testX)
y_pred1[y_pred1==1]=0
y_pred1[y_pred1==-1]=1
y_pred2[y_pred2==1]=0
y_pred2[y_pred2==-1]=1
print ("Score on train set is: ", accuracy_score(valY,y_pred1))
print ("Score for test data is", accuracy_score(testY,y_pred2))
print("Classification report for train set")
print(classification_report(valY,y_pred1))
print("Confusion matrix for train set")
print(confusion_matrix(valY,y_pred1))
print("Confusion matrix for train set")
print(confusion_matrix(testY,y_pred2))
print("Classification report for test set")
print(classification_report(testY,y_pred2))

Score on train set is:  0.9905637604511839
Score for test data is 0.9903093290263685
Classification report for train set
             precision    recall  f1-score   support

          0       1.00      0.99      1.00     45490
          1       0.12      0.73      0.21        79

avg / total       1.00      0.99      0.99     45569

Confusion matrix for train set
[[45081   409]
 [   21    58]]
Confusion matrix for train set
[[56336   528]
 [   24    74]]
Classification report for test set
             precision    recall  f1-score   support

          0       1.00      0.99      1.00     56864
          1       0.12      0.76      0.21        98

avg / total       1.00      0.99      0.99     56962

